This code defines and trains an autoencoder model using the TensorFlow library and the Keras API. An autoencoder is a type of neural network that is used for dimensionality reduction and feature learning.

The autoencoder has an encoder and a decoder part, which are both defined in this code. The encoder part of the model consists of a series of convolutional and max pooling layers that process the input image and extract features from it. The decoder part of the model consists of a series of convolutional and upsampling layers that reconstruct the input image from the encoded representation.

The autoencoder is trained to reconstruct the input image from the encoded representation, and the encoded representation is used as a compact representation of the input image. In this case, the input images are RGB images of size 128x128 pixels, and the autoencoder is trained to minimize the binary cross-entropy loss between the input and the reconstructed images using the Adam optimizer.

The code also defines an ImageDataGenerator object to load and preprocess the images from a directory. The images are divided into a training set and a validation set, and the autoencoder is trained on the training set and evaluated on the validation set.

The code also defines a ModelCheckpoint callback, which saves the weights of the model every 5 epochs during training. Finally, the code fits the autoencoder to the training data using the fit() method, and generates latent representations of the food images using the encoder model.

In [89]:
import tensorflow as tf
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import numpy as np

# Set the input shape and number of classes
INPUT_SHAPE = (128, 128, 3)  # Assume 128x128 RGB images
NUM_CLASSES = 2  # Food or not food

# Set the batch size
BATCH_SIZE = 64

In [90]:
from keras.preprocessing.image import ImageDataGenerator

# Set the directory containing the images
image_dir = 'food_images_2'

# Create an ImageDataGenerator object
image_generator = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Use the generator to load the images from the directory
train_data_gen = image_generator.flow_from_directory(image_dir,
                                                     target_size=(128, 128),
                                                     color_mode='rgb',
                                                     class_mode='binary',
                                                     batch_size=64,
                                                     subset='training')
val_data_gen = image_generator.flow_from_directory(image_dir,
                                                   target_size=(128, 128),
                                                   color_mode='rgb',
                                                   class_mode='binary',
                                                   batch_size=64,
                                                   subset='validation')

# Get the images and labels from the generator
x_train, y_train = next(train_data_gen)
x_val, y_val = next(val_data_gen)


print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)


Found 800 images belonging to 1 classes.
Found 199 images belonging to 1 classes.
(64, 128, 128, 3)
(64,)
(64, 128, 128, 3)
(64,)


In [104]:
# Define the encoder
encoder_input = layers.Input(shape=INPUT_SHAPE)
x = layers.Conv2D(16, 3, activation='relu', padding='same')(encoder_input)
x = layers.MaxPooling2D(2, padding='same')(x)
x = layers.Conv2D(8, 3, activation='relu', padding='same')(x)
x = layers.MaxPooling2D(2, padding='same')(x)
x = layers.Conv2D(8, 3, activation='relu', padding='same')(x)
encoder_output = layers.MaxPooling2D(2, padding='same')(x)

# Define the decoder
decoder_input = encoder_output
x = layers.Conv2D(8, 3, activation='relu', padding='same')(decoder_input)
x = layers.UpSampling2D(2)(x)
x = layers.Conv2D(8, 3, activation='relu', padding='same')(x)
x = layers.UpSampling2D(2)(x)
x = layers.Conv2D(16, 3, activation='relu', padding='same')(x)
x = layers.UpSampling2D(2)(x)
decoder_output = layers.Conv2D(3, 3, activation='sigmoid', padding='same')(x)

# Define the encoder model
encoder = tf.keras.Model(encoder_input, encoder_output)

# Define the autoencoder
autoencoder = tf.keras.Model(encoder_input, decoder_output)



# Compile the autoencoder with a loss function and an optimizer
autoencoder.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))


autoencoder.summary()


Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 conv2d_35 (Conv2D)          (None, 128, 128, 16)      448       
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 64, 64, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_36 (Conv2D)          (None, 64, 64, 8)         1160      
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 32, 32, 8)        0         
 g2D)                                                            
                                                                 
 conv2d_37 (Conv2D)          (None, 32, 32, 8)         584

In [92]:
from keras.callbacks import ModelCheckpoint

NUM_EPOCHS = 5

# Create a ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint('autoencoder_weights.h5', save_weights_only=True, period=5)

# Fit the autoencoder to the training data, using the ModelCheckpoint callback
history = autoencoder.fit(x_train,
                          x_train,
                          epochs=NUM_EPOCHS,
                          batch_size=BATCH_SIZE,
                          validation_data=(x_val, x_val),
                          callbacks=[checkpoint_callback])



Epoch 1/5
1/1 [==============================] - 3s 3s/step - loss: 0.6935 - val_loss: 0.6935
Epoch 2/5
1/1 [==============================] - 1s 806ms/step - loss: 0.6935 - val_loss: 0.6934
Epoch 3/5
1/1 [==============================] - 1s 952ms/step - loss: 0.6934 - val_loss: 0.6934
Epoch 4/5
1/1 [==============================] - 1s 926ms/step - loss: 0.6934 - val_loss: 0.6933
Epoch 5/5
1/1 [==============================] - 1s 849ms/step - loss: 0.6933 - val_loss: 0.6933


In [80]:
from keras.models import Sequential
from keras.layers import Dense


X_encoded = encoder.predict(x_train)

2/2 [==============================] - 0s 31ms/step


In [81]:
from keras.layers import Flatten

# Train a classifier on the lower-dimensional representation of the input images
classifier = Sequential()
classifier.add(Flatten(input_shape=X_encoded.shape[1:]))  # Add a Flatten layer to reshape the input
classifier.add(Dense(32, activation='relu'))
classifier.add(Dense(1, activation='sigmoid'))
classifier.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
classifier.fit(X_encoded, y_train, epochs=3, batch_size=32)

Epoch 1/3
2/2 [==============================] - 1s 4ms/step - loss: 0.4623 - accuracy: 0.6094
Epoch 2/3
2/2 [==============================] - 0s 4ms/step - loss: 0.0057 - accuracy: 1.0000
Epoch 3/3
2/2 [==============================] - 0s 4ms/step - loss: 4.2177e-04 - accuracy: 1.0000


In [103]:
import imageio
import numpy as np
from skimage.transform import resize

# Load the image and convert it to a NumPy array
image_data = imageio.imread('./tripadvisor_images_2/23487213_1.jpg')

# Resize the image to the same size as the training images
resized_image = resize(image_data, (128, 128, 3))

# Add an extra dimension to the array to match the expected input shape of the model
resized_image = np.expand_dims(resized_image, axis=0)

resized_image /= 255.

# Use the encoder to generate a lower-dimensional representation of the image
img_encoded = autoencoder.predict(resized_image)

# Use the classifier to predict the class of the image
prediction = classifier.predict(img_encoded)
print(prediction)  # Will print either [0] or [1]

1/1 [==============================] - 0s 21ms/step
[[0.00018595]
 [0.00018602]
 [0.000186  ]
 [0.00018593]
 [0.00018602]
 [0.000186  ]
 [0.00018594]
 [0.00018602]
 [0.00018601]
 [0.00018594]
 [0.00018603]
 [0.00018601]
 [0.00018594]
 [0.00018602]
 [0.00018601]
 [0.00018594]
 [0.00018602]
 [0.00018601]
 [0.00018594]
 [0.00018602]
 [0.000186  ]
 [0.00018593]
 [0.00018602]
 [0.00018607]]


/var/folders/4l/gsvsrpxd2kqb4f6v1wy203b80000gn/T/ipykernel_5655/1346691681.py:6: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image_data = imageio.imread('./tripadvisor_images_2/23487213_1.jpg')


In [102]:
prediction = np.argmax(prediction)  # Will print either 0 or 1
print(prediction)

0
